In [24]:
import json

import requests
import boto3
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

session = boto3.Session(profile_name="platform-developer")


def _get_secret(secret_name: str):
    secrets_manager_client = session.client('secretsmanager', region_name='eu-west-1')
    response = secrets_manager_client.get_secret_value(SecretId=secret_name)

    return response['SecretString']

# The experimental database is accessible from outside of the VPC via a Network Load Balancer (NLB) for testing purposes
LOAD_BALANCER_URL = _get_secret("NeptuneTest/LoadBalancerUrl")
NEPTUNE_INSTANCE_ENDPOINT = _get_secret("NeptuneTest/InstanceEndpoint")

def run_open_cypher_query(query: str):
    """Run a Cypher query against an experimental serverless Neptune cluster"""
    open_cypher_endpoint_url = f'{LOAD_BALANCER_URL}/openCypher'

    headers = {
        "Host": NEPTUNE_INSTANCE_ENDPOINT,
        "Content-Type": "application/json"
    }
    payload = {"query": query}
    
    # We use IAM database authentication, which means we need to authenticate the request using AWS Signature
    request = AWSRequest(method="POST", url=open_cypher_endpoint_url, data=json.dumps(payload), headers=headers)
    SigV4Auth(session.get_credentials(), "neptune-db", "eu-west-1" ).add_auth(request)

    # We need to send a manual request rather than using boto3 since we are accessing the instance via a NLB
    response = requests.post(
        open_cypher_endpoint_url,
        data=json.dumps(payload),
        headers=dict(request.headers),
        # We are using the default NLB DNS name, which does not support custom SSL certificates, so we need to disable SSL certificate verification.
        # This increases the risks of a man-in-the-middle attack, which is acceptable for a testing database.
        # In production, we will be connecting to the database directly from within the VPC.
        verify=False
    )

    if response.status_code != 200:
        raise Exception(response.content)

    return response.json()

In [ ]:
query = """
CREATE (n:Person {name: 'Alice', age: 30, city: 'New York'})
RETURN n
"""

run_open_cypher_query(query)

In [25]:
query = """
MATCH (n) RETURN n LIMIT 10
"""

run_open_cypher_query(query)

{'results': [{'n': {'~id': '22dcce2d-4577-416a-a07a-11c93b0cf734',
    '~entityType': 'node',
    '~labels': ['Person'],
    '~properties': {'age': 30, 'city': 'New York', 'name': 'Alice'}}}]}